<a href="https://colab.research.google.com/github/ERPGulf/changAI/blob/alpha/changai/notebooks/Synthetic_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install openai pydantic==2.*

import os, json, re, time
from typing import List, Dict, Any
from pydantic import BaseModel, ValidationError
from openai import OpenAI

# Set your key (Colab: Runtime > Secrets or set manually)
from google.colab import userdata
key = userdata.get("OPENAI_API_KEY")

client = OpenAI(api_key=key)

MODEL_BASES   = "gpt-4o"  # good quality; batch gives -50% token price
MODEL_VARIANTS = "gpt-4o"  # you can try a cheaper instruct-compatible model if available


In [ ]:
DOCTYPE   = "Customer"
META_PATH = "/content/meta.json"

def load_fields(doctype: str) -> List[str]:
    with open(META_PATH, "r", encoding="utf-8") as f:
        meta = json.load(f)
    fields = (meta.get(doctype) or {}).get("fields", [])
    return fields
FIELDS = load_fields(DOCTYPE)

class FlanRecord(BaseModel):
    input: Dict[str, Any]
    output: str


In [ ]:
def stage_a_master_prompt(doctype: str, fields: list, used_examples: list) -> str:
    return f"""
You are an expert dataset generator for ERPNext AI assistants.

Task:
For the ERPNext doctype: {doctype}, generate EXACTLY 5 DISTINCT, DIVERSE, and VERY REALISTIC messy business questions that normal ERP users would actually type or say in daily work.
⚠️ IMPORTANT: Do NOT repeat, rephrase, or make similar any question from {used_examples}.

Language style rules:
- Questions must be messy, casual, and sometimes grammatically wrong.
- Use slang, shorthand, or typos (e.g., "cust inv", "cn u show unpaid", "cstmr w/o ph no").
- Include broken grammar, abbreviations, missing words, local-style typing.
- Avoid robotic/professional phrasing.
- No vague placeholders like "some", "specific", "certain".
- Always use concrete realistic filters/values (e.g., "Dubai", "Q2 2024", "last 30 days", "credit > 5k").
- Keep short like real ERP quick queries (6–16 words).

Field rules:
- Use ONLY from: {fields}
- "input.fields" must include EVERY field used in SQL (SELECT, WHERE, GROUP BY, ORDER BY).
- Use 2–8 fields total; always include "name" for row-list queries.

Diversity:
- Each question must be a UNIQUE intent (not just rewording).
- Cover broad functional types: totals/aggregates, overdue/pending, top-N/rankings, by-attribute segmentation, date-range/period, recent/latest, comparisons, anomalies/exceptions, KPI thresholds, activity/changes, full-record retrieval.
- At most 2 can be simple counts.
- Questions MUST feel like real messy queries from non-technical users, not robotic.

Used base questions (NEVER repeat, rephrase, or paraphrase):
{used_examples}

Output format (JSONL only, one per line; no arrays, no prose, no markdown):
{{
  "input": {{
    "doctype": "{doctype}",
    "question": "<messy realistic business question>",
    "fields": ["name", "<other fields used>"]
  }},
  "output": "frappe.db.sql(\\"<VALID SQL query>\", as_dict=True)"
}}
""".strip()


In [ ]:
def write_stage_a_batch_jsonl(path: str, n_calls: int = 10):
    used_examples = [
    "How many customers are in the 'Retail' group?",
    "List customers with no email address",
    "Who are the top 5 customers by revenue?",
    "Who are the top 5 customers by sales in Dubai?",
    "Total number of customers in Dubai?",
    "Show top 5 customers by sales team size",
    "Who are the top 5 customers by sales?",
    "Who are the top 5 customers by sales in Q2 2024?",
    "List all customers by territory in Dubai",
    "What are the top 5 customers by sales team?",
    "Cn u list cust w/o mobile no?",
    "Cust w/o mobile no?",
    "List customers who have 'Technology' as their industry",
    "List customers with missing tax ID",
    "List customers w/ no mobile_no",
    "Top 3 customers by market segment in Q1 2023?",
    "Get cust w/o mob no.",
    "Top 3 customers by loyalty program tier?",
    "How many customers have joined in the last 30 days?",
    "How many customers joined in last 30 days?",
    "How many customers joined in the last month?",
    "How many customers have a loyalty program tier of 'Gold'?",
    "Total customers in the 'Wholesale' group?"
]

    prompt = stage_a_master_prompt(DOCTYPE, FIELDS,used_examples)
    with open(path, "w", encoding="utf-8") as f:
        for i in range(1, n_calls+1):
            item = {
                "custom_id": f"stageA_{DOCTYPE}_{i:04d}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": MODEL_BASES,
                    "messages": [
                        {"role":"system","content":"Output strict JSON only."},
                        {"role":"user","content": prompt}
                    ],
                    "response_format": {"type":"json_object"},
                    "temperature": 0.4,
                    "max_tokens": 2000
                }
            }
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

stageA_in = f"stageA_{DOCTYPE}_inputs_5.jsonl"
write_stage_a_batch_jsonl(stageA_in, n_calls=1)
print("Wrote:", stageA_in)

In [ ]:
up_a = client.files.create(file=open(stageA_in, "rb"), purpose="batch")
batch_a = client.batches.create(
    input_file_id=up_a.id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
)

In [ ]:
print("Stage A batch id:", batch_a.id, " input file id:", up_a.id)
status = client.batches.retrieve(batch_a.id)
print("Status:", status.status)
if status.status == "completed":
    out_id = status.output_file_id
    print("Output file id:", out_id)
content = client.files.content(out_id)
with open("stageA_outputs_5.jsonl", "wb") as f:
    f.write(content.read())
print("Saved results to stageA_outputs_5.jsonl")

In [ ]:
import json
bases = []
with open("/content/Stage1Customer", "r", encoding="utf-8") as f:
  bases = json.load(f)

In [ ]:
seen=set()
results_=[]
for b in bases:
  qstn=b["input"]["question"]
  if qstn not in seen:
    seen.add(qstn)
    results_.append(b)

In [ ]:
def variant_prompt_100(doctype: str, base_q: str) -> str:
    return f"""
Goal: Generate EXACTLY 100 messy/casual variants of this base ERP question (same intent, no new filters/dates/metrics).
BASE: "{base_q}"

Rules:
- Only output a JSON object with key "questions": a list of 100 strings.
- Each string must be a DISTINCT messy/natural variant of the base.
- Use at most ONE synonym for {doctype} Eg : Customer :client,buyer,consumer,shopper,patron,end-user,customer,cust,customer acc,cstmr,byer
- Messiness styles: word-order shuffle, symbols/slashes, shorthand, fillers, telegraphic, hashtags, punctuation bursts, missing punctuation, abbreviations, typos, phonetic spellings, lazy typing.
- Keep realistic, short ERP-style queries (3–18 words, business tone).
- No duplicates, no placeholders, no paraphrases that look robotic.

Output format (strict JSON):
{{
  "questions": [
    "variant 1",
    "variant 2",
    ...
    "variant 100"
  ]
}}
""".strip()


def write_stage_b_batch_jsonl(bases: list, stageB_in_path: str): # Accept bases list as argument
    with open(stageB_in_path, "w", encoding="utf-8") as w: # Write to stageB_in_path
        for i, b in enumerate(results_, start=1):
            base_q = b["input"]["question"]
            item = {
                "custom_id": f"stageB_{DOCTYPE}_{i:04d}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": MODEL_VARIANTS,
                    "messages": [
                        {"role":"system","content":"Output strict JSON only."},
                        {"role":"user","content": variant_prompt_100(DOCTYPE, base_q)}
                    ],
                    "response_format": {"type":"json_object"},
                    "temperature": 0.9,
                    "max_tokens": 1800
                }
            }
            w.write(json.dumps(item, ensure_ascii=False) + "\n")
    print("Wrote:", stageB_in_path, "items:", len(results_))

# Example (after parsing Stage A):
stageB_in = f"stageB_{DOCTYPE}_inputs_1.jsonl"
write_stage_b_batch_jsonl(results_, stageB_in) # Pass the loaded bases list

In [ ]:
up = client.files.create(file=open(stageB_in, "rb"), purpose="batch")

In [ ]:
up_a = client.files.create(file=open(stageB_in, "rb"), purpose="batch")
batch_a = client.batches.create(
    input_file_id=up_a.id,
    endpoint="/v1/chat/completions",
    completion_window="24h"
)

In [ ]:
print("Batch ID:", batch_a.id)

In [ ]:
print("Stage A batch id:", batch_a.id, " input file id:", up_a.id)
status = client.batches.retrieve(batch_a.id)
print("Status:", status.status)
if status.status == "completed":
    out_id = status.output_file_id
    print("Output file id:", out_id)
content = client.files.content(out_id)
with open("stageB_outputs.jsonl", "wb") as f:
    f.write(content.read())
print("Saved results to stageB_outputs.jsonl")

In [ ]:
with open("S1Customer_no_dupes","w",encoding="utf-8") as f:
  json.dump(results_,f,ensure_ascii=True,indent=2)

In [ ]:
print(results_)

In [ ]:
bases=[]
with open("/content/stageA_outputs_1.jsonl", "r", encoding="utf-8") as f:
  for line in f:
    if line.strip():
        bases.append(json.loads(line))
        print(bases[:1])

In [ ]:
import json

results = []

with open("/content/stageA_outputs_1.jsonl","r",encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        content = obj.get("response", {}) \
                     .get("body", {}) \
                     .get("choices", [{}])[0] \
                     .get("message", {}) \
                     .get("content", "")

        if content:
            try:
                parsed = json.loads(content)   # convert JSON string -> dict
                records = parsed.get("records", [])
                results.extend(records)        # add all records
            except json.JSONDecodeError:
                print("Invalid JSON:", content[:100])  # debug

print("Total records:", len(results))
print(results[:3])  # show first 3 records


In [ ]:
with open("/content/Stage1Customer","w",encoding="utf-8") as f:
  json.dump(results,f,ensure_ascii=True,indent=2)


In [ ]:
results_[0]["output"]

In [ ]:
with open("/content/stageB_outputs.jsonl","r",encoding="utf-8") as f:
  for line in f:
    line=line.strip()
    if not line:
      continue
    obj=json.loads(line)
    content=obj.get("response",{}) \
                  .get("body",{}) \
                  .get("choices",[{}])[0] \
                  .get("message",{}) \
                  .get("content","")

    for i in range(len(results_)):
      if content:


In [ ]:
with open("/content/stageB_outputs.jsonl","r",encoding="utf-8") as f:
  out=[]

  for line in f:
    line=line.strip()
    if not line:
      continue
    obj=json.loads(line)
    content=obj.get("response",{}) \
                  .get("body",{}) \
                  .get("choices",[{}])[0] \
                  .get("message",{}) \
                  .get("content","")
    if content:
            out.append({
              "input": {"doctype": results_[i]["input"]["doctype"], "question": content, "fields": results_[i]["input"]["fields"]},
              "output": results_[i]["output"]
          })
